<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_bow_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the modules used
import numpy as np
import scipy as sci
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [2]:
# For reproducible results
# except for variability introduced by GPU
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '42'
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # avoid using GPU for reproducible results
np.random.seed(42)
rn.seed(42)
# tf.random.set_seed(42)

In [3]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [4]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [5]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [6]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

In [7]:
# model 1: BOW + logistic model
# dense representation is not imposed by logistic regression
# sparse representation is needed so that logistic regression does not run out of memory
cv = CountVectorizer(strip_accents='unicode', lowercase=True, stop_words='english', dtype=np.int8) 
cv_train_sparse = cv.fit_transform(train_text)
# cv_train_dense = sci.sparse.csr_matrix.todense(cv_train_sparse)

cv_test_sparse = cv.transform(test_text)
# cv_test_dense = sci.sparse.csr_matrix.todense(cv_test_sparse)
# print('BOW for cv_train:', cv_train_dense.shape)
# print('BOW for cv_test:', cv_test_dense.shape)

In [8]:
type(cv_test_sparse)

scipy.sparse.csr.csr_matrix

In [9]:
MAX_ITER = 100
model1 = LogisticRegression(max_iter=MAX_ITER)
model1.fit(cv_train_sparse, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
pred_soft1 = model1.predict(cv_test_sparse)

In [11]:
# pred = np.around(pred_soft, decimals = 0)
pred1 = np.where(pred_soft1 > 0.15, 1, 0)

acc1 = accuracy_score(pred1, test_labels)
f11 = f1_score(pred1, test_labels)

cm1 = confusion_matrix(test_labels, pred1)
tn1 = cm1[0][0]
fn1 = cm1[1][0]
tp1 = cm1[1][1]
fp1 = cm1[0][1]

print('Accuracy score: {:.4f}'.format(acc1), 'F1 score: {:.4f}'.format(f11))
print('False Positives: {:.0f}'.format(fp1), 'False Negatives: {:.0f}'.format(fn1))
print('Confusion matrix:\n', cm1)

Accuracy score: 0.9858 F1 score: 0.8538
False Positives: 5 False Negatives: 33
Confusion matrix:
 [[2533    5]
 [  33  111]]


In [12]:
# sanity check the model parameters
print (type(model1.coef_), model1.coef_.shape[0], model1.coef_.shape[1])
print (model1.coef_, model1.intercept_)

<class 'numpy.ndarray'> 1 96150
[[-9.65592381e-02  5.02190765e-01 -9.68099635e-03 ... -1.04612747e-06
  -1.04612747e-06 -1.04612747e-06]] [-1.58213478]


In [13]:
# print the top-k words with the largest coefficients
# these words contribute the most to a job description declared fraudulent
coef = model1.coef_.reshape(model1.coef_.shape[1])
k = 20
ind = np.argpartition(coef, -k)[-k:]
for i in range (k):
  print(ind[i], coef[ind[i]], list(cv.vocabulary_.keys())[list(cv.vocabulary_.values()).index(ind[i])])

33685 0.583666549778054 forward
90601 0.5880771388570726 warsaw
50275 0.5922453583229651 migration
58424 0.5894725690060855 personable
25611 0.6014009057248951 duration
3537 0.6200141194192783 administrative
85478 0.6097342380566507 trends
2560 0.6230277267508464 accountant
51410 0.6342872633589646 motivated
51049 0.6430812625819265 money
8060 0.6540206821739426 au
25832 0.7034693991546753 earn
2420 0.7909873621854235 accion
28288 0.7036004048677369 entry
90433 0.696350398561821 wages
70901 1.2517113334375831 rohan
6562 0.8661454561535944 aptitude
53381 0.6744264060592281 northwestern
45978 1.0131995590683975 link
4869 0.6697178300384368 american
